In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [5]:
data1 = pd.read_csv("math_expr_new.csv",index_col=None)
data = data1.iloc[73000:78000,].copy()

In [6]:
data.columns=['Math Expresion','Expression Name']

In [7]:
data

,Math Expresion,Expression Name
73000,\frac{-8}{3} x-6,linear_expression
73001,-7.7\left(9x+\frac{4}{7}\right)-5.4\left(\frac...,distributor_expression
73002,2+9x,linear_expression
73003,\frac{5}{6} \left(\frac{4}{3}-4x\right)+7\left...,linear_equation
73004,-4x+3+\frac{2}{9} x+0.9=2\left(9x+5\right),linear_equation
...,...,...
77995,6\left(-x+7.4\right)=\frac{6}{5} x+\frac{8}{9},linear_equation
77996,-2.8x-\frac{2}{5}-6.3+\frac{7}{9} x=\frac{-2}{...,linear_equation
77997,-0.7\left(-4x-0.6\right)=3.2x-1.9,linear_equation
77998,-9x+3-2.7x-3,linear_expression


In [8]:
data.loc[data["Expression Name"]=='distributor_expression','Expression Name']=0
data.loc[data["Expression Name"]=='linear_expression','Expression Name']=1
data.loc[data["Expression Name"]=='linear_equation','Expression Name']=2

In [9]:
a=0
for s in data.iloc[:,0]:
    data.iloc[a,0] = bytearray(s, 'utf-8')
    a+=1
    # print(data.iloc[a,0][0]+"--- {}s seconds ---".format(time.time() - start_time, 2))

In [10]:
X_train,X_test,y_train,y_test = train_test_split(data.iloc[:,0],data.iloc[:,1],test_size=0.2)
##print(X_train.shape)
##print(X_test.shape)
##print(y_train.shape)
##print(y_test.shape)

In [11]:
X_train_seq = sequence.pad_sequences(X_train, maxlen=200)
X_test_seq = sequence.pad_sequences(X_test, maxlen=200)

In [12]:
Y_train_ctg = np_utils.to_categorical(y_train)
Y_test_ctg = np_utils.to_categorical(y_test)

In [13]:
earlystop = EarlyStopping(monitor='val_loss', mode='min',min_delta=0, verbose = 1, patience=5)
mc = ModelCheckpoint('best_model_5000.h5', monitor='val_loss', mode='min', save_best_only=True)

In [14]:
model = Sequential()
model.add(Embedding(5000, 100))
model.add(LSTM(100, activation='tanh'))
model.add(Dense(3, activation='softmax'))

In [15]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
history = model.fit(X_train_seq, Y_train_ctg, batch_size=128, 
                    epochs=50,validation_data=(X_test_seq, Y_test_ctg),callbacks=[earlystop,mc])

C:\Users\boinit\anaconda3\envs\myenv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4000 samples, validate on 1000 samples
Epoch 1/50
4000/4000 [==============================] - 7s 2ms/step - loss: 0.8646 - accuracy: 0.6743 - val_loss: 0.5768 - val_accuracy: 0.7610
Epoch 2/50
4000/4000 [==============================] - 6s 2ms/step - loss: 0.4716 - accuracy: 0.7648 - val_loss: 0.3845 - val_accuracy: 0.8120
Epoch 3/50
4000/4000 [==============================] - 6s 1ms/step - loss: 0.3219 - accuracy: 0.8547 - val_loss: 0.3106 - val_accuracy: 0.8920
Epoch 4/50
4000/4000 [==============================] - 4s 1ms/step - loss: 0.1318 - accuracy: 0.9630 - val_loss: 0.0595 - val_accuracy: 0.9910
Epoch 5/50
4000/4000 [==============================] - 4s 1ms/step - loss: 0.0277 - accuracy: 0.9970 - val_loss: 0.0237 - val_accuracy: 0.9980
Epoch 6/50
4000/4000 [==============================] - 4s 1ms/step - loss: 0.0178 - accuracy: 0.9980 - val_loss: 0.0207 - val_accuracy: 0.9980
Epoch 7/50
4000/4000 [==============================] - 4s 1ms/step - loss: 0.0141 - acc

In [28]:
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test_seq, Y_test_ctg)[1]))

1000/1000 [==============================] - 1s 1ms/step

 Test Accuracy: 0.8880


In [32]:
model.save('./models/MExC_lstm_V1.h5')

In [33]:
print(history.history['loss'])
#print(history.history['acc'])
print(history.history['val_loss'])
print(history.history['val_accuracy'])

[0.8646241898536682, 0.4716026537418366, 0.3219049929380417, 0.131842261955142, 0.02770541549101472, 0.017830320075154303, 0.014081969372928143, 0.01044783964008093, 0.00886562688741833, 0.010860043905675411, 0.00911318263411522, 0.006132641468197107, 0.004143273098859936, 0.0026690817922353743, 0.001579288886860013, 0.0010955453594215215, 0.0008526926897466183, 0.000698735476937145, 0.0005792972953058779, 0.0004934097076766193, 0.0004275336505379528, 0.0003800346963107586, 0.00033678308909293263, 0.0002976743918843567, 0.0002701675579883158, 0.00024582303024362775, 0.00022284824773669243, 0.00020506793330423535, 0.00018782596383243799, 0.00017322213051375, 0.00015883192932233215, 0.00014765598298981786, 0.00013784437789581715, 0.00012843872263329105, 0.00012031404324807227, 0.00011311799532268196, 0.00010699440585449338, 0.8726725717422087, 0.54490274143219, 0.38238441467285156, 0.3621607446670532, 0.2676178286075592]
[0.5767853274345398, 0.38450086617469786, 0.31061383390426633, 0.05

In [20]:
a=history.history['loss']
b=history.epoch
c=history.history['val_loss']

In [21]:
min(history.history['val_loss'])

0.00011861226265318691

In [22]:
result = pd.DataFrame({'학습데이터량':a,
                       'Epoch' : b,
                       '손실최소값':c})

In [23]:
result

,학습데이터량,Epoch,손실최소값
0,0.864624,0,0.576785
1,0.471603,1,0.384501
2,0.321905,2,0.310614
3,0.131842,3,0.059515
4,0.027705,4,0.023735
5,0.017830,5,0.020670
6,0.014082,6,0.035593
7,0.010448,7,0.014718
8,0.008866,8,0.012333
9,0.010860,9,0.014834


In [26]:
result.to_csv('aaa.csv',encoding='euc-kr')